# Importation

In [1]:
## Installation et Import des packages necessaires (pandas, Nltk)
!pip3 install --user -U numpy
!pip3 install --user -U nltk
!pip3 install pandas

## Import de packages necessaires pour la suite
import nltk
import re
import math
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import pandas as pd 
import xml.etree.ElementTree as et 
from nltk.corpus import wordnet
import nltk
import difflib
import glob
import string 
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords 
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet 
lem = WordNetLemmatizer() 
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()
nltk.download('sentiwordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')

Requirement already up-to-date: numpy in /home/deptinfo/.local/lib/python3.8/site-packages (1.19.5)
Requirement already up-to-date: nltk in /home/deptinfo/.local/lib/python3.8/site-packages (3.5)


[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/deptinfo/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/deptinfo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/deptinfo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/deptinfo/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/deptinfo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/deptinfo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Def des fonctions 

## Traitements des docuements :

In [2]:
#Fct qui va parcourir tout les .txt d'un repertoir pour copier leur contenu dans une liste
#la fct prends en argument le chemin/nom du repertoir cible
#Retourne une liste ou chaque indice est un document
def txt_doc_to_string_list(doc):
    doc_list = glob.glob(doc + "/*.txt")
    txt_list = []
    doc_list = sorted(doc_list)
    for i in range(len(doc_list)):
        with open(doc_list[i], 'r') as file:
            txt_list.append(file.read().replace('\n', ' '))
    return txt_list

In [3]:
#Tokenise tout les documents avec nltk
def string_to_token(txt_list):
    txt_list_token = []
    for i in range(len(txt_list)):
        doc_token = nltk.word_tokenize((txt_list[i].lower()))
        txt_list_token.append(doc_token)
    return txt_list_token

In [4]:
#FCt qui supprime la ponctuation, les stop word et les adresses https
def remove_punctuation_stopwords(txt_list_token):
    stop_words = set(stopwords.words('english'))
    listword = ["_","-","(",")","—","‘",'’','”',"https","http"]
    for doc in txt_list_token:
        for word in doc[::-1]: 
            if word in string.punctuation or word in listword or "//" in word :
                doc.remove(word)
            if word in stop_words:
                doc.remove(word)
    return txt_list_token

In [5]:
#Lemmatise tout les documents contenue dans la liste en fonction du leurs tag
def lemma_text(docs):
    lem_docs = []
    for doc in docs:
        lem_doc = []
        doc_tag = nltk.pos_tag(doc)
        for word in doc_tag:
            pos_label = (word[1][0]).lower()
            if pos_label == 'j': pos_label = 'a'    
            if pos_label in ['a', 's', 'v']:
                lem_word = lem.lemmatize(word[0], pos=pos_label)
            else:   
                lem_word = lem.lemmatize(word[0])
            lem_doc.append(lem_word)
        lem_docs.append(lem_doc)
    return lem_docs

## Fct qui génère le dictionnaire 

In [6]:
#Génère un dictionnaire a partire des documents lemmatisé 
#Pour cela, je récupérer chaque mots, j'indique dans quel document il apparait et sa fréquence par doc
def to_dic(lemma):
    dic = pd.DataFrame(columns=['word','document','frequence'])
    for i in range(len(lemma)):
        list_word = list(dict.fromkeys(lemma[i])) 
        for word in list_word:
            nb_occurance = lemma[i].count(word)
            dic = dic.append({'word' : word ,'document' : i ,'frequence' : nb_occurance }, ignore_index=True)
    return dic

## Fct qui gènre la matrice 

In [27]:
#Génère la matrice a aprtir du dictionnaitre
#la matrice a pour colonne les numero des documents
#SI le mot apparait dans le document j'ajoute un "1" dans la matrice, sinon un 0
def matrice(df) :
    #all_word = df['word'].tolist()
    all_word = list(dict.fromkeys(df['word'].tolist())) 
    col = list(dict.fromkeys(df["document"].tolist()))
    matrice = pd.DataFrame(columns=col, index = all_word)
    for word in all_word:
        liste = list(df.loc[df['word']==word]["document"])
        for val in liste :
            matrice.loc[word, val] = 1
    #matrice = matrice.fillna(0)
    return matrice.fillna(0)

## Fct qui génère les index

In [8]:
#Génère les index a aprtir du dictionnaire
#Chaque mot sera dans un vecteur ou il y aura simplement le numero du docuement ou il apparait 
def index_inverse(df):
    vect_words = []
    all_word = list(dict.fromkeys(df['word'].tolist())) 
    for i in range(len(all_word)):
        vect_words.append([all_word[i],list(df.loc[df['word']==all_word[i]]["document"])])
    return vect_words

# Requetes

## Requetes Booléennes

In [9]:
#Traite les requetes booleen (tokenisation, lemmatisation, distinction de si on demande un and, un and not ou un or)
def traitement_requete_bool(requete):
    token_req =  nltk.word_tokenize(requete.lower())
    req_tag = nltk.pos_tag(token_req)
    lem_req = []
    for word in req_tag:
        pos_label = (word[1][0]).lower()
        if pos_label == 'j': pos_label = 'a'
        if pos_label in ['a', 's', 'v']:
            lem_word = lem.lemmatize(word[0], pos=pos_label)
        else:   # For nouns and everything else as it is the default kwarg
            lem_word = lem.lemmatize(word[0])
        lem_req.append(lem_word)
    req  = []
    res = []
    search = []
    andnot = []
    str_req = ' '.join(lem_req)
    str_req = str_req.replace("( ","")
    str_req = str_req.replace(" )","")
    if " and not " in str_req:
        tab = str_req.split(" and not ")
        search = tab [0]
        andnot = tab [1]
    else :
        search = str_req
    search = search.split(" and ")

    if len(andnot) > 0 :
        andnot = andnot.split(" and ")
        for i in range(len(andnot)):
            if " or " in andnot[i]:
                andnot[i] = andnot[i].split(" or ")
            if " " in andnot[i]:
                tab = andnot[i].split(" ")
                for word in tab:
                    andnot.append(word)
                andnot.pop(i)
    for i in range(len(search)):
        if " or " in search[i] :
            search[i] = search[i].split(" or ")
        if " " in search[i]:
                tab = search[i].split(" ")
                for word in tab:
                    search.append(word)
                search.pop(i)
    return andnot,search        

In [10]:
#Applique une requete simple qui récupérer les documents contenant les mots désirés et les classe avec la fréquence des mots
def res_requete_bool(req_bool,df):
    #Trainement de la requete booléenne
    tab_req = traitement_requete_bool(req_bool)
    andnot = tab_req[0]
    andword = tab_req[1]

    wordnot = []
    valnot =[]
    for pos in andnot:
        if isinstance(pos, list):
            tab = df[df["word"].isin(pos)]
            valnot = tab.document.unique().tolist()
      
        else :
            wordnot.append(pos)
    tabnot = df[df["word"].isin(wordnot)]
    tabnot2 = tabnot[tabnot.groupby("document")["document"].transform('size') == len(wordnot)]
    valnot= valnot + tabnot2.document.unique().tolist()


    word = []
    val = []
    sum_val = []
    for pos in andword:
        if isinstance(pos, list):
            tab2 = df[df["word"].isin(pos)]
            #val = tab2.document.unique().tolist()
            docs = tab2.groupby("document")['frequence'].sum().index.values.tolist()
            freq = tab2.groupby("document")['frequence'].sum().values.tolist()
            for i in range(len(docs)):
                sum_val.append([docs[i],freq[i]])

        else :
            word.append(pos)
    tabword = df[df["word"].isin(andword)]
    #tabword2 = tabword[tabword.groupby("document")["document"].transform('size') == len(word)]
    docs = tabword.groupby("document")['frequence'].sum().index.values.tolist()
    freq = tabword.groupby("document")['frequence'].sum().values.tolist()

    for i in range(len(docs)):

        sum_val.append([docs[i],freq[i]])
    index = []
    for i in range(len(sum_val)):
        for val in valnot:
            if val == sum_val[i][0]:
                index.append(i)
    for val in index[::-1]:
        sum_val.pop(val)

    res = pd.DataFrame(sum_val,columns =['document', 'freq'])

    res = res.groupby("document")['freq'].sum().to_frame().sort_values(by ='freq', ascending=False )
    
    
    return res

# Requetes complexes

In [11]:
#Traitement des requete complexe (tokenisation, stop word, lemmatisation..)
def traitement_req_complexe(req_comp):
    stop_words = set(stopwords.words('english'))
    listword = ["_","-","(",")","—","‘",'’','”',"https","http"]


    req =  nltk.word_tokenize(req_comp.lower())
    for word in req[::-1]: 
        if word in string.punctuation or word in listword or "//" in word :
            req.remove(word)
        if word in stop_words:
            req.remove(word)

    req_tag = nltk.pos_tag(req)
    lem_req = []

    for word in req_tag:
        pos_label = (word[1][0]).lower()
        if pos_label == 'j': pos_label = 'a'
        if pos_label in ['a', 's', 'v']:
            lem_word = lem.lemmatize(word[0], pos=pos_label)
        else:   # For nouns and everything else as it is the default kwarg
            lem_word = lem.lemmatize(word[0])
        lem_req.append(lem_word)
    return lem_req

In [12]:
#Calcule du TF*IDF
#Cette fct ne prends qu'un terme et non une requete entière
#Retourne le TFIDF du termes en fonction des documents

def tfidf(df, matrice,docs,req):
    nb_doc = len(matrice.columns)
    nb_doc_terme = matrice.sum(axis=1)
    tabword = df[df["word"] == req ]
    wordfreq = 0
    for word, freq in nb_doc_terme.items():
        if word == req:
            wordfreq = freq
    tabword = tabword.drop(columns=['word'])        
    records = tabword.to_records(index=False)
    result = list(records)

    res_cal = []
    for i in range(len(result)):
        res = (result[i][1]/len(docs[i]))*(math.log((nb_doc/wordfreq),2)+1)
        res_cal.append([result[i][0],res])
    return res_cal

In [13]:
def classement_doc_req_comp(df,matrice,docs,req_tr):
    #Trainement de la requete complexe
    req= traitement_req_complexe(req_tr)
    columns = []
    df_ = pd.DataFrame(index=matrice.columns, columns=columns)
    for i in range(len(req)):
        tab = tfidf(df, matrice,docs,req[i])
        for j in range(len(tab)):
            df_.loc[tab[j][0], req[i]] = tab[j][1]
    #Je supprime les lignes contenant des NaN car cela veut dire que ces documents ne contiennent pas tout les mots requis
    #df_ = df_.dropna()
    classement = df_.sum(axis=1).to_frame("freq").sort_values(by ='freq', ascending=False )
    classement = classement[classement.freq > 0.0]
    return classement

# Application des fct réalisées plus haut 

## Traitement du document 

In [14]:
#FCt qui regroupe toutes les fct précédente qui traitent le document donc il y a la tokenisation, la suppresion de la ponctuation, stop word, lemmatisation...
def traitement_doc(nomdoc):
    #Extraction des txt dans le repertoire documents pour les convertirs en une liste de String (la list contient tout les documents)
    txtlist = txt_doc_to_string_list("documents")
    #Transformation en token
    txtlist_token = string_to_token(txtlist)
    #Suppréssion des stopword et de la punctuation
    txtlist_token_nostop = remove_punctuation_stopwords(txtlist_token)
    #Lemmatisation
    lemma_txt = lemma_text(txtlist_token_nostop)
    
    return lemma_txt

## Récupération de la liste des documents avec des mots lemmatisé et génération du dictionnaire, de la matrice et des index

In [28]:
#Traitement des documents en retournant une liste contenant la liste des mots lemmatisé par document
lemma = traitement_doc("documents")
#Dictionnaire relatif a la liste des documents
dic_txt = to_dic(lemma)
#Matrice d'incidence a partir du dictionnaire
matrice_incidence = matrice(dic_txt)
#Index inverse a partir du dictionnaire
ind_inverse = index_inverse(dic_txt)

## Application de la fonction qui classe les documents en fonction du de la frequence des mots  (requete booléenne)
### J'ai déjà écrit les requetes de l'énoncé, il vous suffira de changer le numero de la requete dans les arguements de la fonction "res_requete_bool"

In [29]:
##Requete booléenne de l'énoncé :
requete_bool1 = "desease AND severe AND pneumonia"
requete_bool2 = "antibody AND plasma AND (cells OR receptors)"
requete_bool3 = "antimalarial drugs OR antiviral agents OR immunomodulators"
requete_bool4 = "NOT plasma AND risk of infection AND restrictions"
requete_bool5 = "(older adults AND antibodies) AND NOT (genomes OR variant)"

#df contenant tout les documents répondant a la requete, du meilleur au plus mauvais
doc_freq_bool = res_requete_bool(requete_bool2,dic_txt) ## Changez le premier argument de cette fonction en fonction avec la requete que vous voulez tester
doc_freq_bool

traitement_requete_bool(requete_bool5 )

([['genome', 'variant']], ['antibody', 'old', 'adult'])


## Application de la fonction qui classe les documents en fonction tf*idf  (requete complexe)
### J'ai déjà écrit les requetes de l'énoncé, il vous suffira de changer le numero de la requete dans les arguements de la fonction "res_requete_bool"

In [17]:
##Requete complexe de l'énoncé :
req_comp1 = "antibody treatments"
req_comp2 = "efficacy and safety of the treatments"
req_comp3 = "family access to hospitals"
req_comp4 = "contact tracing results"
req_comp5 = "genomic analysis of SARS-CoV-2 disease"

#df contenant tout les documents répondant a la requete, du meilleur au plus mauvais
doc_freq_comp = classement_doc_req_comp(dic_txt,matrice_incidence,lemma,req_comp3)  ## Changez le dernier argument de cette fonction en fonction avec la requete que vous voulez tester
doc_freq_comp

,freq
8,0.085381
7,0.040799
1,0.007646
2,0.004776
0,0.003754
6,0.001793
9,0.001539
3,0.001425
4,0.000494


# Pour finir vous pouvez visualiser le dictionnaire, la matrice et les index :

## Dictionnaire :

In [18]:
dic_txt

,word,document,frequence
0,13,0,12
1,january,0,14
2,mutation,0,43
3,undercut,0,1
4,immune,0,103
...,...,...,...
9512,beta-coronaviruses,9,1
9513,reveal,9,1
9514,apply,9,1
9515,variety,9,1


## Matrice d'incidence :

In [19]:
matrice_incidence

,0,1,2,3,4,5,6,7,8,9
13,1,1,1,1,0,0,0,0,1,0
january,1,1,0,0,0,0,0,0,0,0
mutation,1,0,0,0,0,0,0,0,0,0
undercut,1,0,0,0,0,0,0,0,0,0
immune,1,0,1,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
interplay,0,0,0,0,0,0,0,0,0,1
cross-immunity,0,0,0,0,0,0,0,0,0,1
alpha-,0,0,0,0,0,0,0,0,0,1
beta-coronaviruses,0,0,0,0,0,0,0,0,0,1


## list des Index :

In [20]:
ind_inverse

[['13', [0, 1, 2, 3, 8]],
 ['january', [0, 1]],
 ['mutation', [0]],
 ['undercut', [0]],
 ['immune', [0, 2, 3, 6, 9]],
 ['response', [0, 2, 3, 4, 7, 8, 9]],
 ['covid', [0, 7, 9]],
 ['virus', [0, 1, 2, 3, 4, 5, 9]],
 ['handful', [0]],
 ['sars-cov-2', [0, 1, 2, 3, 4, 5, 6, 8, 9]],
 ['help', [0, 1, 2, 4]],
 ['escape', [0, 9]],
 ['mount', [0]],
 ['subset', [0]],
 ['infect', [0, 1, 3, 9]],
 ['people', [0, 5, 7, 9]],
 ['researcher', [0, 4, 8]],
 ['identify', [0, 1, 2, 3, 8]],
 ['thousand', [0]],
 ['sample', [0, 1, 3, 4, 8]],
 ['vast', [0]],
 ['majority', [0, 4, 6]],
 ['unlikely', [0, 3]],
 ['much', [0, 7, 9]],
 ['effect', [0, 1, 2, 3, 4, 7, 8, 9]],
 ['biology', [0]],
 ['potentially', [0, 1, 7, 8, 9]],
 ['important', [0, 1, 3, 4, 8, 9]],
 ['jesse', [0]],
 ['bloom', [0]],
 ['fred', [0]],
 ['hutchinson', [0]],
 ['cancer', [0, 2]],
 ['research', [0, 2, 3, 4, 8]],
 ['center', [0, 1, 2, 3, 4, 5]],
 ['seattle', [0]],
 ['washington', [0]],
 ['colleague', [0, 8]],
 ['study', [0, 1, 2, 3, 4, 6, 7, 8, 9